In [1]:
import pandas as pd

In [2]:
pd.options.display.max_columns = 100

In [3]:
# csv 파일 불러오기
df_2024 = pd.read_csv('./data/서울특별시_전월세가_2024.csv', encoding='cp949')
df_2023 = pd.read_csv('./data/서울특별시_전월세가_2023.csv')

In [5]:
# 1인가구 면적 필터링
df_2024 = df_2024[df_2024['임대면적(㎡)'] <= 35].reset_index(drop=True).copy()
df_2023 = df_2023[df_2023['임대면적'] <= 35].reset_index(drop=True).copy()

In [6]:
# 임대료 0원 제거
df_2024 = df_2024[df_2024['임대료(만원)'] != 0].reset_index(drop=True).copy()
df_2023 = df_2023[df_2023['임대료(만원)'] != 0].reset_index(drop=True).copy()

In [14]:
# 불필요한 컬럼 제거
df_2024 = df_2024[['접수연도', '자치구명', '전월세 구분', '임대면적(㎡)', '보증금(만원)', '임대료(만원)', '건물용도', '신규갱신여부']].copy()
df_2023 = df_2023[['접수년도', '자치구명', '전월세구분', '임대면적', '보증금(만원)', '임대료(만원)', '건물용도', '신규계약구분']].copy()

In [15]:
# 컬럼명 전체 변경
df_2024.columns = [['접수년도', '자치구명', '전월세구분', '임대면적', '보증금', '임대료', '건물용도', '신규갱신구분']]
df_2023.columns = [['접수년도', '자치구명', '전월세구분', '임대면적', '보증금', '임대료', '건물용도', '신규갱신구분']]

In [16]:
# 결측값 제거
df_2024 = df_2024.dropna().reset_index(drop=True)
df_2023 = df_2023.dropna().reset_index(drop=True)

In [9]:
# 돈 단위 변경
df_2024['보증금'] = df_2024['보증금'].apply(lambda x: x * 10000)
df_2024['임대료'] = df_2024['임대료'].apply(lambda x: x * 10000)
df_2023['보증금'] = df_2023['보증금'].apply(lambda x: x * 10000)
df_2023['임대료'] = df_2023['임대료'].apply(lambda x: x * 10000)

In [10]:
# 데이터프레임 행으로 합치기
merge_df = pd.concat([df_2023, df_2024])

In [11]:
# index 초기화
merge_df.reset_index(drop=True, inplace=True)

In [41]:
import pymysql

con = pymysql.connect(host='', user='', password='', db='', charset='utf8')
cur = con.cursor()

In [42]:
sql = 'insert into allRent values (%s, %s, %s, %s, %s, %s, %s, %s)'

In [43]:
from tqdm import tqdm

for idx, row in tqdm(merge_df.iterrows()):
    cur.execute(sql, row.tolist())

con.commit()

160498it [23:26, 114.11it/s]


In [ ]:
# DB 데이터 불러오기
lh_df = pd.read_sql_query('select * from allLh', con=con)
ansim_df = pd.read_sql_query('select * from ansim', con=con)

In [ ]:
# geocoder 2.0 API로 위도, 경도 데이터 찾기
import requests

tmp = ''
data = []
failed = []

for idx, row in tqdm(lh_df.iterrows()):
    apiurl = "https://api.vworld.kr/req/address?"
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": row['adresWay'],
        "format": "json",
        "type": "road",
        "key": "[API-KEY]"
    }
    response = requests.get(apiurl, params=params)
    if response.status_code == 200:
        tmp = response.json()
        try:
            data.append(tmp['response']['result']['point'])
        except Exception as e:
            params["type"] = "parcel"
            response = requests.get(apiurl, params=params)
            if response.status_code == 200:
                tmp = response.json()
                try:
                    data.append(tmp['response']['result']['point'])
                except Exception as e:
                    failed.append(idx)

In [ ]:
# 위도 경도 데이터 저장
for i in range(len(lh_df)):
    lh_df.loc[i, 'x'] = data[i]['x']
    lh_df.loc[i, 'y'] = data[i]['y']

In [ ]:
# CSV 파일 저장
lh_df.to_csv('lh_df.csv', index=False)
ansim_df.to_csv('ansim_df.csv', index=False)